In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

sns.set_context('talk')

In [2]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [4]:
def remove_outlier(df_in, col_name, iqr_factor):
    """
  IQR outliers removal (https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/)
  accept a dataframe, remove outliers, return cleaned data in a new dataframe
  see http://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
  """
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1  # interquartile range
    fence_low = q1 - iqr_factor * iqr
    fence_high = q3 + iqr_factor * iqr
    df_inliers = df_in.loc[
        (df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)
    ]
    df_outliers = df_in.loc[
        (df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)
    ]
    return df_inliers, df_outliers

In [6]:
iqr_factor = 2.5
df_main_inliers, df_main_outliers = remove_outlier(df, "salary_monthly_NETO", iqr_factor)
df_main_inliers = df_main_inliers[df_main_inliers["salary_monthly_NETO"] > 24000]
# Filtramos por el mínimo vital y móvil

display(len(df_main_inliers) / len(df))


0.8720262510254306

In [8]:
df_main_inliers[['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                                       \
                             count          mean           std      min   
profile_gender                                                            
Hombre                      4464.0  85788.666004  39867.724585  24500.0   
Mujer                        826.0  74645.487397  33861.570050  25000.0   
Otros                         25.0  80260.000000  44573.628227  34000.0   

                                                      
                    25%      50%       75%       max  
profile_gender                                        
Hombre          56000.0  80000.0  105000.0  240000.0  
Mujer           49800.0  68000.0   92978.5  230000.0  
Otros           40000.0  74000.0  101000.0  190000.0

In [9]:
alpha = 0.05

In [21]:

groupA = df_main_inliers[df_main_inliers['profile_gender']=='Hombre']
groupB = df_main_inliers[df_main_inliers['profile_gender']!='Hombre']

groupB['salary_monthly_NETO'].mean()

74810.42607520563

In [26]:
muA, stdA = stats.poisson.fit(groupA['salary_monthly_NETO'])

muB, stdB = stats.poisson.fit(groupB['salary_monthly_NETO'])
print(muA, stdA)
print()
print(muB, stdB)

AttributeError: 'poisson_gen' object has no attribute 'fit'

In [42]:
dif_mean=groupA.mean() - groupB.mean()
nA=groupA['salary_monthly_NETO'].count()-1
nB=groupB['salary_monthly_NETO'].count()-1


850